In [1]:
library('tidyverse', warn.conflicts = FALSE)
library('robustbase')


── Attaching core tidyverse packages ────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ──────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [12]:
master <- "/Users/laurituominen/Documents/Research/Reettis/old_new_analyses/master/"
dat <- read_csv(paste0(master, 'merged_data.csv'),show_col_types = FALSE)

dat <- dat %>% mutate(Sex = factor(Sex, levels = c('M', 'F')),
                      group =factor(Group, levels=c('CTR', 'CHR', 'FEP')))

# exposure vs cortical thickness 
model <- dat %>% filter(group %in% c('FEP', 'CHR')) %>% 
  lmrob(mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group, na.action=na.omit, data=.)

model %>% summary()


Call:
lmrob(formula = mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group, 
    data = ., na.action = na.omit)
 \--> method = "MM"
Residuals:
      Min        1Q    Median        3Q       Max 
-0.193423 -0.060353 -0.002418  0.060400  0.197570 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           2.636e+00  3.093e-02  85.221  < 2e-16 ***
Lifetime_ap_exposure -1.128e-06  2.711e-07  -4.159 5.88e-05 ***
Age                  -2.814e-03  1.070e-03  -2.630   0.0096 ** 
SexF                 -3.191e-02  1.449e-02  -2.202   0.0295 *  
GroupFEP             -1.975e-02  1.528e-02  -1.293   0.1985    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Robust residual standard error: 0.08595 
Multiple R-squared:  0.183,	Adjusted R-squared:  0.1571 
Convergence in 9 IRWLS iterations

Robustness weights: 
 15 weights are ~= 1. The remaining 116 ones are summarized as
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5765  0.8926 

In [17]:
model %>% confint()

print(paste('residuals:', model$df.residual))

,2.5 %,97.5 %
(Intercept),2.574812e+00,2.697237e+00
Lifetime_ap_exposure,-1.664085e-06,-5.909500e-07
Age,-4.931578e-03,-6.966632e-04
SexF,-6.059479e-02,-3.229555e-03
GroupFEP,-4.997944e-02,1.048614e-02


[1] "residuals: 126"


In [54]:
# covariates in sensitivity analyses 
covars <- c("Total_symptoms_score", "Positive_symptoms_score", "Negative_symptoms_score", "BMI", "Hospital_days", "Times_admitted",
            "GAF", "SOFAS")

# estimate the effects including one of the sensitivity covariates in the model at the time 
vals <- matrix(nrow=length(covars), ncol=7 )
for (i in seq(from=1,to=length(covars))){
  formula <- reformulate(termlabels =  c("Lifetime_ap_exposure","Age","Sex","Group", covars[i]), response = 'mean_thickness')
  m <- dat %>% filter(group %in% c('FEP', 'CHR')) %>%
    lmrob(formula = formula, data=.)
  tmp <- c(m$df.residual, as.vector(summary(m)$coefficients[2,]), as.vector(confint(m)[2,]))    
  vals[i,] <- tmp # get parameters for "Lifetime_ap_exposure"
}

# make printable 
df <- as.data.frame(vals)
df$covariate <- covars 
df <- df[,c(8,2,6,7,1, 4,5)]
names(df) <- c("Covariate", "Estimate", "2.5% quantile", "97.5% quantile", "df", "t-value", "p-value")
# print out the parameters for "Lifetime_ap_exposure" of each of the sensitivity model 
df #%>% format(scientific=TRUE)

Covariate,Estimate,2.5% quantile,97.5% quantile,df,t-value,p-value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Total_symptoms_score,-1.196205e-06,-1.693172e-06,-6.992372e-07,120,-4.765708,5.329216e-06
Positive_symptoms_score,-1.200869e-06,-1.717961e-06,-6.837776e-07,120,-4.598097,1.063677e-05
Negative_symptoms_score,-1.206886e-06,-1.699264e-06,-7.145075e-07,120,-4.853078,3.694999e-06
BMI,-1.088512e-06,-1.586438e-06,-5.905862e-07,125,-4.326548,3.069204e-05
Hospital_days,-1.040095e-06,-1.662274e-06,-4.179164e-07,124,-3.308760,1.226916e-03
Times_admitted,-1.050679e-06,-1.610185e-06,-4.911726e-07,124,-3.716827,3.039683e-04
GAF,-1.163543e-06,-1.722611e-06,-6.044757e-07,125,-4.118996,6.869324e-05
SOFAS,-1.164073e-06,-1.714481e-06,-6.136661e-07,125,-4.185711,5.317290e-05
